# Лаборатоная работа 2. Получение и анализ ультразвуковой записи.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from IPython.display import display
import ipywidgets as widgets
from PIL import Image

Для осуществления процедуры оцифровки полученных в ходе практической работы экспериментальных ультразвуковых записей в данной работе будет использоватся библиотека компьютерного зрения **cv2**. 

Ссылка на тестовый видеофай: https://disk.yandex.ru/i/TXyTJx-uznzFnQ
Видеофайл должен лежать в папке с файлом.

In [2]:
# Анализ видеофайла с использованием библиотеки компьютерного зрения CV2
# Функция VideoCapture принимает на вход имя видеофайла, находящегося в одной директории с программой

us_video = cv2.VideoCapture('Сагайдачный2003.mkv')

# Если имя файла указано верно, следуюший алгоритм вернёт информацию о количестве и частоте кадров

if us_video.isOpened() == False:
    print('Ошибка открытия файла')
else:
    fps = int(us_video.get(cv2.CAP_PROP_FPS))
    print("Частота кадров: ", fps, "кадров в секунду") 
    frame_count = int(us_video.get(cv2.CAP_PROP_FRAME_COUNT))
    print('Всего кадров: ', frame_count)
    durat = frame_count/fps
    print('Длительность видео: ', durat, ' секунд или ', durat/60, ' минут' )


Частота кадров:  60 кадров в секунду
Всего кадров:  73260
Длительность видео:  1221.0  секунд или  20.35  минут


Процедура воспроизведения видеозаписи с использованием библиотеки компьютерного зрения
Параметр file counter позволяет определить количество воспроизводимых кадров в цикле

In [3]:
file_count = 0
while(us_video.isOpened()):
            # Метод vid_capture.read() возвращают кортеж, первым элементом является логическое значение
            # а вторым кадр
    ret, frame = us_video.read()
    # Данный цикл позволяет вывести 600 кадров видеозаписи, что при 60 к/с даёт нам 10 секунд
    if ret == True and file_count <= 600:
        cv2.imshow('Video', frame)
        file_count += 1
        key = cv2.waitKey(20)    
        # Инициализация кнопок q и Esc (кодируется 27) для осуществления прерывания воспроизведения видеофайла
        if (key == ord('q')) or key == 27:
            break
    else:
            break

# Освободить объекты
us_video.release()
cv2.destroyAllWindows()

Сохранение интересующей нас видеозаписи как последовательность кадров

In [4]:
def FrameSave(counter, pathName, vidName):
    file_count = 0
    us_video = cv2.VideoCapture(vidName)
    try:
        os.mkdir(pathName)
    except FileExistsError:
        pass
    if us_video.isOpened() == False:
        print('Ошибка открытия файла')
    else:
        pass
        while(us_video.isOpened()):
            ret, frame = us_video.read()
            if ret == True and file_count <= counter:
                file_count += 1
                key = cv2.waitKey(1)
                fname = f'usVid_{file_count:0004}.png'
                direct = pathName+'\\'+ fname
                cv2.imwrite(direct, frame)
            else:
                break
    us_video.release()
    cv2.destroyAllWindows()

In [5]:
# Пример взаимодействия с функцией FrameSave 
passName = 'image_sequence5'
videoName = 'Сагайдачный2003.mkv'
Count = 50
FrameSave(Count, passName, videoName)

"passName = 'image_sequence4'\nvideoName = 'Сагайдачный2003.mkv'\nCount = 50\nFrameSave(Count, passName, videoName)"

Обработка видеофайла целиком не является оптимальным. 
Главной задачей обработки длительных ультразвуковых данных является поиск параметров избражения, которые однозначно позволят различить два уникальных изображения и провести оцифровку всей записи.

На первом изображении представлен участок ультразвуковой записи на котором белой рамкой отмечен маркер обновления экрана. Пока этот маркер на экране, изображение не может считаться уникальным, так как на нем остается информация о предыдущем уникальном кадре.

<img src='usVid_0001.jpg' width=960, heigth=720>

На втором представленном кадре отсутствует маркер обновления экрана, что позволяет нам выделить данный кадр как уникальный. Набор уникальных последовательных кадров позволит нам провести полную оцифровку экспериментального сигнала.

<img src='usVid_0228.jpg' width=960, heigth=720>

In [6]:
#  Поиск уникальных кадров для проведения процедуры оцифровки
#  Функция uniqueFrame позволит определить является ли кадр ультразвуковой записи уникальным
# и хранит ли он значимую информациюю.
#  На вход функция принимает адрес файла, на выходе она говорит Уникальный или Неуникальный

def uniqueFrame(paht_in_loc):
    pict = cv2.cvtColor(cv2.imread(paht_in_loc), cv2.COLOR_BGR2GRAY)
    subIm1 = pict[450:710, 440:1614]
    SubMin = np.min(subIm1[177, :])
    SubSign = np.sign(SubMin-100)
    if SubSign >= 0:
        print('Это уникальный кадр')
    else:
        print('Это неуникальный кадр')


In [7]:
directionIn = "image_sequence4\\usVid_0001.png"
uniqueFrame(directionIn)


Это неуникальный кадр


In [8]:
# Реализация алгоритма постороения оцифрованного сигнала

nameUniqueWindow = "image_sequence_unique\\usVid_0225.jpg" # Анализируемое изображение
UniqueWindow  = cv2.cvtColor(cv2.imread(nameUniqueWindow), cv2.COLOR_BGR2GRAY)
subImage = UniqueWindow[450:710, 440:1614] # Границы в которые укладывается экспериментальный сигнал во всех уникальных кадрах

# Функция, реализующая алгоритм оцифровки экспериментального сигнала.
# На вход принимается изображение и порог для фильтрации, на выходе формируется оцифрованная кривая.
%matplotlib widget
def CifrGrapfWoP(data_loc, lEdit_loc):
    Thresh = int(lEdit_loc) # Ожидаемый порог по яркости, отделяющий спетр от фона
    Graph = data_loc[:] # Переменная содержащяя изображение как двумерный массив
    TGraph = np.transpose(Graph) # Переменная содержащяя изображение как двумерный массив в транспонированном виде
    N = [] # Создание массива огибающей максимума
    M = [] # Создание массива огибающей минимума
    F = 0 #
    V = 0 #
    for i in range(len(TGraph)):
        for j in range(len(Graph)):
            if Graph[j, i] > Thresh:
                F = j
                break
        N.append(F * -1)
    MaxR = len(Graph)-1
    for i in range(len(TGraph)):
        for j in range(len(Graph)):
            if Graph[MaxR - j, i] > Thresh:
                V = j
                break
        M.append((MaxR - V) * -1)
    Top = np.asarray(N)
    Bot = np.asarray(M)
    Result = Top + Bot
    return Result

# Функция связывающая нажатие кнопки и выполнение кода
def clicked(arg):
    with out:
        out.clear_output()
        CifrMass = CifrGrapfWoP(subImage, text_widget.value)
        fig = plt.figure(figsize=(10, 3))
        ax = fig.add_subplot(1,2,1)
        ax.set_title("Ультразвуковой спектр", size = 10)
        ax.imshow(subImage, cmap = 'gray', aspect='auto')
        ax = fig.add_subplot(1,2,2)
        ax.set_title(f"Оцифровка с порогом  {text_widget.value} точек", size = 10)
        ax.plot(CifrMass)
        plt.show()
        print(f'Порог оцифровки равен {text_widget.value}')
           
out = widgets.Output(layout={'border': '1px solid black'})
display(out)
button = widgets.Button(description = 'Test Button')   
button.on_click(clicked)
text_widget = widgets.IntText(
    value=100,
    description='Порог:',
    disabled=False
)
display(button, text_widget)

#Попробуйте найти значение порога, при котором получится наилучшая оцифрованная кривая

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

Button(description='Test Button', style=ButtonStyle())

IntText(value=100, description='Порог:')

# Задания ко второй лабораторной работе.

1. Сохранить экспериментальную видеозапись как набор 600 кадров c использованием функции FrameSave;
2. Применить функцию UniqueFrame для определения уникальности кадра ко всему массиву кадрированной экспериментальной записи и реализуйте аналогично функции FrameSave операцию сохранения уникальных кадров;
3. Оцифровать все уникальные кадры, применив функцию CifrGrapfWoP в цикле, и получить единую кривую;
4. Сформировать экспериметальную оцифрованную запись максимальной скорости с сохранением исходных скоростных параметров (осуществить масштабирование амплитуды сигнала);
5. Определить максимумы и минимумы записи.